# REP2SI Regular Graph Extractor

### Problem statement
* Given a directed graph `G`, find a subgraph `H` that is a directed regular graph (equal in-degree and out-degree) of degree `k` with all the same nodes as `G`.

### General approach
* Some edges in `G` are more important to include (even necessary) in `H` than others in order to find a valid solution.
* We assign each edge in `G` a score, based on the desired degree `k`, the current in/out-degree of the nodes it connects in `H`, and the in/out degree in `G`.
* Add edges from `G` to `H`, roughly in order of decreasing score (with some added randomness / reassignment to avoid getting stuck with non-solutions), recalculating scores after each step, until `H` is a regular graph of degree `k`.

### Demo implementation

* Generate some random graphs
* First graphs are generated by creating a regular graph of degree `j` and then adding edges such that each node has out-degree `l` according to preferential attachment.
* Next graphs are generated by preferential attachment where each node has out-degree `l`, but with rewiring such that each node has minimum in-degree `j`. Here there is not a guarantee that a regular subgraph exists.
* The function `extract_regular_subgraph` takes a graph `G` and a desired degree `k` and attempts to find a subgraph `H` of `G` that is a regular graph of degree `k`.
* The function `extract_regular_subgraph` is called on each of the generated graphs, for `k` between 1 and `j/2`, and results stored.

In [1]:
from subgrapher import *

# Generate some random graphs

# generate regular digraph first, add edges by PA after
G1 = regular_digraph_plus_pa(100, 5, 10)
G2 = regular_digraph_plus_pa(400, 10, 30)

# generate PA with outdegree fixed, rewire to get min in-degree
G3 = random_network_fixed_outdegree(100, 10, 5)
G4 = random_network_fixed_outdegree(400, 30, 10)

In [2]:
# dictionaries with graphs, to record succcess/failure, and to store results
original_graphs = {'regular_pa_100_5':G1, 'regular_pa_400_10':G2, 'random_fixed_100_5': G3,
          'random_fixed_400_10':G4}
success = {'regular_pa_100_5':{}, 'regular_pa_400_10':{}, 'random_fixed_100_5': {},
            'random_fixed_400_10':[]}
results = {'regular_pa_100_5':{}, 'regular_pa_400_10':{}, 'random_fixed_100_5': {},
            'random_fixed_400_10':{}}

# iterate through graphs
# this takes a while on the larger graphs and larger n, be patient / edit as necessary
for k, g in original_graphs.items():
    print('Graph %s:' %k, g.number_of_nodes(), g.number_of_edges())
    # extract subgraphs of increasing degree (up to half the minimum in-degree)
    for n in range(1, min(dict(g.in_degree()).values())//2 + 1):
        print('Extracting subgraph of fixed node degree %d from %s' %(n, k))
        try:
            subgraph = extract_regular_subgraph(g, n) # extract subgraph
            success[k][n] = True # record success
            results[k][n] = subgraph # store subgraph
        except Exception as e: # if extraction fails, record failure
            print(e)
            success[k][n] = False
            results[k][n] = None

print(success)


Graph regular_pa_100_5: 100 1000
Extracting subgraph of fixed node degree 1 from regular_pa_100_5
Shuffling 1 at 1.00%. 96/100 edges assigned
Shuffling 2 at 1.01%. 96/100 edges assigned
Shuffling 3 at 1.02%. 96/100 edges assigned
Shuffling 4 at 1.03%. 96/100 edges assigned
Shuffling 5 at 1.04%. 96/100 edges assigned
Shuffling 6 at 1.05%. 96/100 edges assigned
Shuffling 7 at 1.06%. 96/100 edges assigned
Shuffling 8 at 1.07%. 96/100 edges assigned
Shuffling 9 at 1.08%. 96/100 edges assigned
Shuffling 10 at 1.00%. 96/100 edges assigned
Shuffling 11 at 1.01%. 96/100 edges assigned
Shuffling 12 at 1.02%. 96/100 edges assigned
Shuffling 13 at 1.03%. 96/100 edges assigned
Shuffling 14 at 1.04%. 96/100 edges assigned
Shuffling 15 at 1.05%. 96/100 edges assigned
Shuffling 16 at 1.06%. 96/100 edges assigned
Shuffling 17 at 1.07%. 96/100 edges assigned
Shuffling 18 at 1.00%. 98/100 edges assigned
Shuffling 19 at 1.01%. 98/100 edges assigned
Shuffling 20 at 1.02%. 98/100 edges assigned
Shuffling 2

### Potential improvements

* Derive mathematical condition for possible existnce of a regular subgraph.
* Seems to struggle with `k > 3` and/or `k > j/2`.
* Could try to find a better way to assign scores to edges, considering future graph structure (Monte Carlo?).
* Could be smarter on which edges to remove when shuffling (/'backtracking'), some edges may preclude a solution so could permanently exclude them.
* Amount of shuffling and rate of increase could be tuned.
* Can be slow on larger graphs and larger `k`, mostly to do with shuffling repeats.